In [3]:
import os
import json
import glob
import numpy as np
import torch

from NSM.mesh import create_mesh
import sys 
sys.path.append('/dataNAS/people/aagatti/projects/nsm_femur/generate_shapes/python_calls')



In [ ]:
os.environ['LOC_SDF_CACHE'] = '/dataNAS/people/aagatti/projects/deepsdf_femur/cache' # SET THIS LATER FOR CACHEING DATA
from recon_utils import load_model, get_recon_results

RUN_NAME = '231_nsm_femur_cartilage_v0.0.1'
SPLIT = 'test' # 'train', 'val', 'test'
CHECKPOINT_EPOCH = 2000

FOLDER_SAVED_CHECKPOINTS = '/dataNAS/people/aagatti/projects/nsm_femur/training_run_files/experiment_results'
FOLDER_SAVED = '/dataNAS/people/aagatti/projects/nsm_femur/saved_predictions/CVPR'
FOLDER_QUEUES = '/dataNAS/people/aagatti/projects/nsm_femur/generate_shapes/queues'
LOG_FILENAME = 'recon_log.txt'

# Path model weights
path_model_state = os.path.join(FOLDER_SAVED_CHECKPOINTS, RUN_NAME, 'model', f'{CHECKPOINT_EPOCH}.pth')

# Path model Config
path_model_config = os.path.join(FOLDER_SAVED_CHECKPOINTS, RUN_NAME, 'model_params_config.json')

# LOAD CONFIG
with open(path_model_config, 'r') as f:
    model_config = json.load(f)

model = load_model(model_config, path_model_state, model_type='triplanar')


folder_saved = os.path.join(FOLDER_SAVED, RUN_NAME, f'{CHECKPOINT_EPOCH}_{SPLIT}')
folder_latents = os.path.join(folder_saved, 'latents')
list_latents = glob.glob(os.path.join(folder_latents, '*_latent.npy'))

[KeOps] Compiling cuda jit compiler engine ... 
[KeOps] Warning : There were warnings or errors compiling formula :
/usr/bin/ld: cannot find -lnvrtc: No such file or directory
collect2: error: ld returned 1 exit status

OK
[pyKeOps] Compiling nvrtc binder for python ... 
[KeOps] Warning : There were warnings or errors compiling formula :
/usr/bin/ld: cannot find -lnvrtc: No such file or directory
collect2: error: ld returned 1 exit status

OK


In [5]:
ref_latent = np.load(list_latents[0]).squeeze()


torch_vec = torch.tensor(ref_latent, dtype=torch.float).cuda()
mesh = create_mesh(
    model, 
    torch_vec, 
    objects=2
)

del torch_vec
torch.cuda.empty_cache() 

fem_mesh, cart_mesh = mesh
print(f'fem mesh orig n points: {fem_mesh.point_coords.shape}')
fem_mesh.resample_surface(clusters=20000)
orig_pts = fem_mesh.point_coords.copy()

fem mesh orig n points: (153042, 3)


In [6]:
from NSM.mesh.interpolate import interpolate_mesh

In [ ]:
folder_save = '/dataNAS/people/aagatti/projects/nsm_femur/dense_correspondence/test_shapes'


In [8]:
base_mesh = fem_mesh.copy()

new_latent = np.load(list_latents[np.random.randint(1, len(list_latents))]).squeeze()
new_mesh = interpolate_mesh(model, ref_latent, new_latent, n_steps=100, mesh=base_mesh.copy(), surface_idx=0, verbose=False, spherical=True, max_edge_len=0.04, adaptive=False, smooth=True, smooth_type='laplacian')
new_mesh.save_mesh(os.path.join(folder_save, 'interp_mesh_laplace.vtk'))

In [11]:
base_mesh.save_mesh(os.path.join(folder_save, 'interp_mesh_laplace_orig.vtk'))

In [14]:
new_mesh = interpolate_mesh(model, ref_latent, new_latent, n_steps=100, mesh=base_mesh.copy(), surface_idx=0, verbose=False, spherical=True, max_edge_len=0.04, adaptive=True, smooth=True, smooth_type='laplacian')
new_mesh.save_mesh(os.path.join(folder_save, 'interp_mesh_laplace_adaptive.vtk'))



In [15]:
new_mesh = interpolate_mesh(model, ref_latent, new_latent, n_steps=1000, mesh=base_mesh.copy(), surface_idx=0, verbose=False, spherical=True, max_edge_len=0.04, adaptive=True, smooth=True, smooth_type='laplacian')
new_mesh.save_mesh(os.path.join(folder_save, 'interp_mesh_laplace_adaptive_1k.vtk'))



In [16]:
new_mesh = interpolate_mesh(model, ref_latent, new_latent, n_steps=1000, mesh=base_mesh.copy(), surface_idx=0, verbose=False, spherical=True, max_edge_len=0.04, adaptive=False, smooth=True, smooth_type='laplacian')
new_mesh.save_mesh(os.path.join(folder_save, 'interp_mesh_laplace_1k.vtk'))